In [1]:
from vllm import LLM, SamplingParams
model_name = "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"
# model_name = "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa"

llm = LLM(
            model=model_name,
            dtype="half"
)

sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=500
)

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-31 14:42:51,245	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 03-31 14:42:51 __init__.py:207] Automatically detected platform cuda.
WARNING 03-31 14:42:52 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-31 14:43:01 config.py:549] This model supports multiple tasks: {'embed', 'reward', 'score', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 03-31 14:43:01 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', speculative_config=None, tokenizer='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quanti

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.39s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.57it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.33it/s]



INFO 03-31 14:43:05 model_runner.py:1115] Loading model weights took 4.7199 GB
INFO 03-31 14:43:06 worker.py:267] Memory profiling takes 0.51 seconds
INFO 03-31 14:43:06 worker.py:267] the current vLLM instance can use total_gpu_memory (44.40GiB) x gpu_memory_utilization (0.90) = 39.96GiB
INFO 03-31 14:43:06 worker.py:267] model weights take 4.72GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 34.69GiB.
INFO 03-31 14:43:06 executor_base.py:111] # cuda blocks: 28419, # CPU blocks: 3276
INFO 03-31 14:43:06 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 111.01x
INFO 03-31 14:43:08 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:12<00:00,  2.87it/s]

INFO 03-31 14:43:20 model_runner.py:1562] Graph capturing finished in 12 secs, took 0.31 GiB
INFO 03-31 14:43:20 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 14.87 seconds


In [2]:
query = "sort a list in python"
prompts = [f"Expand the following search query with additional relevant terms and make 20 expanded queries. Seperate each expanded query with '\n' seperator. \nQuery: {query}\nExpanded Queries:"]

results = llm.generate(prompts, sampling_params)
explanations = [result.outputs[0].text for result in results]

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it, est. speed input: 16.56 toks/s, output: 110.90 toks/s]


In [3]:
explanations

['\n1. sort a list in python using built-in sort function\n2. sort a list in python using sorted() function\n3. sort a list in python using lambda function\n4. sort a list in python using custom sort function\n5. sort a list in python using key function\n6. sort a list in python using reverse order\n7. sort a list in python using multiple keys\n8. sort a list in python using custom key function\n9. sort a list in python using custom comparison function\n10. sort a list in python using custom sorting algorithm\n11. sort a list in python using heapq module\n12. sort a list in python using bisect module\n13. sort a list in python using heapq.nsmallest() function\n14. sort a list in python using heapq.nlargest() function\n15. sort a list in python using sorted() function with key and reverse\n16. sort a list in python using list.sort() method with key and reverse\n17. sort a list in python using sorted() function with custom comparison function\n18. sort a list in python using list.sort() 

In [ ]:
# Ensure you have installed the required packages:
# pip install rank-bm25 nltk

from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re

nltk.download('punkt')


splitted_explanations = [line.strip() for line in explanations[0].split('\n') if line.strip()]

splitted_explanations = [re.sub(r'^\d+\.\s*', '', exp) for exp in splitted_explanations]

original_query = "sort a list in python"

tokenized_explanations = [word_tokenize(exp.lower()) for exp in splitted_explanations]
tokenized_query = word_tokenize(original_query.lower())

bm25 = BM25Okapi(tokenized_explanations)

scores = bm25.get_scores(tokenized_query)
print("BM25 Scores for candidate explanations:", scores)

best_index = np.argmax(scores)
best_explanation = explanations[best_index]

print("\nOriginal Query:", original_query)
print("Best Explanation:", best_explanation)


BM25 Scores for candidate explanations: [1.18711262 1.047546   1.15146349 1.18711262 1.15146349 1.15146349
 1.15146349 1.09704935 1.09704935 1.09704935 1.15146349 1.15146349
 1.047546   1.047546   0.88737757 0.88737757 0.88737757 0.88737757
 1.047546   1.047546  ]

Original Query: sort a list in python
Best Explanation: 
1. sort a list in python using built-in sort function
2. sort a list in python using sorted() function
3. sort a list in python using lambda function
4. sort a list in python using custom sort function
5. sort a list in python using key function
6. sort a list in python using reverse order
7. sort a list in python using multiple keys
8. sort a list in python using custom key function
9. sort a list in python using custom comparison function
10. sort a list in python using custom sorting algorithm
11. sort a list in python using heapq module
12. sort a list in python using bisect module
13. sort a list in python using heapq.nsmallest() function
14. sort a list in python

[nltk_data] Downloading package punkt to
[nltk_data]     /home/anamikaghosh_umass_edu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
